# Loop 5 Analysis: Comprehensive Ensemble from All Sources

## Key Insight from jonathanchan kernel:
1. Ensemble from 19+ sources (GitHub, Kaggle datasets, notebooks)
2. Override N=1 with optimal value (x=0, y=0, deg=45)
3. Apply fractional translation (0.001 to 0.00001 steps)

## Available Sources:
- 88 snapshots (already scanned)
- Preoptimized folder: telegram, santa25-public, bucket-of-chump, etc.
- 3434 total CSV files

In [1]:
import pandas as pd
import numpy as np
import os
import glob
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import math
from tqdm import tqdm
import json

# Tree geometry
def make_tree_polygon():
    tw, th = 0.15, 0.2
    bw, mw, ow = 0.7, 0.4, 0.25
    tip, t1, t2, base, tbot = 0.8, 0.5, 0.25, 0.0, -0.2
    x = [0, ow/2, ow/4, mw/2, mw/4, bw/2, tw/2, tw/2, -tw/2, -tw/2, -bw/2, -mw/4, -mw/2, -ow/4, -ow/2]
    y = [tip, t1, t1, t2, t2, base, base, tbot, tbot, base, base, t2, t2, t1, t1]
    return list(zip(x, y))

TREE_TEMPLATE = make_tree_polygon()

def get_tree_polygon(x, y, deg):
    poly = Polygon(TREE_TEMPLATE)
    poly = rotate(poly, deg, origin=(0, 0))
    poly = translate(poly, x, y)
    return poly

def score_group(xs, ys, degs):
    n = len(xs)
    all_x, all_y = [], []
    for i in range(n):
        r = math.radians(degs[i])
        c, s = math.cos(r), math.sin(r)
        for tx, ty in TREE_TEMPLATE:
            X = c * tx - s * ty + xs[i]
            Y = s * tx + c * ty + ys[i]
            all_x.append(X)
            all_y.append(Y)
    side = max(max(all_x) - min(all_x), max(all_y) - min(all_y))
    return side * side / n

def check_overlaps(xs, ys, degs):
    n = len(xs)
    polys = [get_tree_polygon(xs[i], ys[i], degs[i]) for i in range(n)]
    for i in range(n):
        for j in range(i+1, n):
            if polys[i].intersects(polys[j]) and not polys[i].touches(polys[j]):
                inter = polys[i].intersection(polys[j])
                if inter.area > 1e-10:  # Significant overlap
                    return True, f"Trees {i} and {j} overlap (area={inter.area})"
    return False, "OK"

print("Functions defined")

Functions defined


In [2]:
# Find ALL CSV files in preoptimized folder
preopt_base = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/'

preopt_files = []
for root, dirs, files in os.walk(preopt_base):
    for f in files:
        if f.endswith('.csv'):
            preopt_files.append(os.path.join(root, f))

print(f"Found {len(preopt_files)} preoptimized CSV files")
for f in preopt_files[:20]:
    print(f"  {f}")

Found 30 preoptimized CSV files
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/ensemble.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/submission.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/best_ensemble.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/72.49.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/71.97.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/telegram_extracted/72.49.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/telegram/telegram_extracted/71.97.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/submission_JKoT4.csv
  /home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa25-public/New_Tree_144_196.csv
  /home/nonroot/snapshots/santa-2025/21116303805/co

In [3]:
# Score each preoptimized file
def strip_s(val):
    s = str(val)
    return float(s[1:] if s.startswith('s') else s)

def load_and_score_csv(filepath):
    try:
        df = pd.read_csv(filepath)
        if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
            return None, None
        df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
        
        scores = {}
        for n, g in df.groupby('N'):
            if n < 1 or n > 200:
                continue
            xs = [strip_s(v) for v in g['x'].values]
            ys = [strip_s(v) for v in g['y'].values]
            ds = [strip_s(v) for v in g['deg'].values]
            scores[n] = score_group(xs, ys, ds)
        
        total = sum(scores.values())
        return total, scores
    except Exception as e:
        return None, None

# Score all preoptimized files
preopt_scores = {}
for fp in tqdm(preopt_files, desc="Scoring preoptimized"):
    total, per_n = load_and_score_csv(fp)
    if total is not None:
        preopt_scores[fp] = {'total': total, 'per_n': per_n}

print(f"\nScored {len(preopt_scores)} files")

# Sort by total score
sorted_preopt = sorted(preopt_scores.items(), key=lambda x: x[1]['total'])
print("\nTop 10 preoptimized solutions:")
for fp, data in sorted_preopt[:10]:
    print(f"  {data['total']:.6f}: {os.path.basename(fp)}")

Scoring preoptimized:   0%|          | 0/30 [00:00<?, ?it/s]

Scoring preoptimized:   3%|▎         | 1/30 [00:00<00:03,  8.18it/s]

Scoring preoptimized:   7%|▋         | 2/30 [00:00<00:03,  8.32it/s]

Scoring preoptimized:  10%|█         | 3/30 [00:00<00:03,  7.71it/s]

Scoring preoptimized:  13%|█▎        | 4/30 [00:00<00:03,  8.00it/s]

...

Scoring preoptimized:  93%|█████████▎| 28/30 [00:03<00:00,  8.67it/s]

Scoring preoptimized:  97%|█████████▋| 29/30 [00:03<00:00,  8.84it/s]

Scoring preoptimized: 100%|██████████| 30/30 [00:03<00:00,  8.36it/s]

Scoring preoptimized: 100%|██████████| 30/30 [00:03<00:00,  8.57it/s]


Scored 30 files

Top 10 preoptimized solutions:
  70.676102: ensemble.csv
  70.676102: santa-2025.csv
  70.676102: best_ensemble.csv
  70.676102: santa-2025.csv
  70.676501: submission.csv
  70.676501: submission.csv
  70.926150: submission_70_926149550346.csv
  70.926150: submission_best.csv
  70.936674: submission_70_936673758122.csv
  70.990692: submission_opt1.csv


In [4]:
# Now let's build the BEST ensemble from ALL sources
# 1. Load all snapshot submissions
# 2. Load all preoptimized files
# 3. For each N, find the best VALID solution

# First, let's load the current valid baseline
valid_baseline_path = '/home/nonroot/snapshots/santa-2025/21328309254/submission/submission.csv'
baseline_df = pd.read_csv(valid_baseline_path)
baseline_df['N'] = baseline_df['id'].astype(str).str.split('_').str[0].astype(int)

baseline_per_n = {}
for n, g in baseline_df.groupby('N'):
    xs = [strip_s(v) for v in g['x'].values]
    ys = [strip_s(v) for v in g['y'].values]
    ds = [strip_s(v) for v in g['deg'].values]
    baseline_per_n[n] = {
        'score': score_group(xs, ys, ds),
        'data': g.drop(columns=['N']).copy(),
        'source': 'valid_baseline'
    }

print(f"Baseline total: {sum(d['score'] for d in baseline_per_n.values()):.6f}")

Baseline total: 70.647327


In [5]:
# Build best-per-N from ALL sources (including preoptimized)
best_per_n = {n: baseline_per_n[n].copy() for n in range(1, 201)}

# Add preoptimized solutions
for fp, data in tqdm(preopt_scores.items(), desc="Processing preoptimized"):
    try:
        df = pd.read_csv(fp)
        df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
        
        for n, g in df.groupby('N'):
            if n < 1 or n > 200:
                continue
            xs = [strip_s(v) for v in g['x'].values]
            ys = [strip_s(v) for v in g['y'].values]
            ds = [strip_s(v) for v in g['deg'].values]
            score = score_group(xs, ys, ds)
            
            if score < best_per_n[n]['score']:
                best_per_n[n] = {
                    'score': score,
                    'data': g.drop(columns=['N']).copy(),
                    'source': os.path.basename(fp)
                }
    except:
        continue

# Calculate total score (ignoring overlaps for now)
total_best = sum(d['score'] for d in best_per_n.values())
print(f"\nBest ensemble (ignoring overlaps): {total_best:.6f}")
print(f"Improvement from baseline: {sum(d['score'] for d in baseline_per_n.values()) - total_best:.6f}")

Processing preoptimized:   0%|          | 0/30 [00:00<?, ?it/s]

Processing preoptimized:   3%|▎         | 1/30 [00:00<00:03,  8.42it/s]

Processing preoptimized:   7%|▋         | 2/30 [00:00<00:03,  8.34it/s]

Processing preoptimized:  10%|█         | 3/30 [00:00<00:03,  7.62it/s]

Processing preoptimized:  13%|█▎        | 4/30 [00:00<00:03,  7.89it/s]

...

Processing preoptimized:  93%|█████████▎| 28/30 [00:03<00:00,  8.48it/s]

Processing preoptimized:  97%|█████████▋| 29/30 [00:03<00:00,  8.65it/s]

Processing preoptimized: 100%|██████████| 30/30 [00:03<00:00,  8.13it/s]

Processing preoptimized: 100%|██████████| 30/30 [00:03<00:00,  8.43it/s]


Best ensemble (ignoring overlaps): 70.647327
Improvement from baseline: 0.000000


In [6]:
# Now check which solutions have overlaps
print("Checking for overlaps in best solutions...")

overlap_ns = []
for n in tqdm(range(1, 201), desc="Checking overlaps"):
    g = best_per_n[n]['data']
    xs = [strip_s(v) for v in g['x'].values]
    ys = [strip_s(v) for v in g['y'].values]
    ds = [strip_s(v) for v in g['deg'].values]
    
    has_overlap, msg = check_overlaps(xs, ys, ds)
    if has_overlap:
        overlap_ns.append(n)
        # Revert to baseline
        best_per_n[n] = baseline_per_n[n].copy()

print(f"\nFound {len(overlap_ns)} N values with overlaps: {overlap_ns[:20]}...")

# Calculate valid total
valid_total = sum(d['score'] for d in best_per_n.values())
print(f"\nValid ensemble score: {valid_total:.6f}")
print(f"Improvement from baseline: {sum(d['score'] for d in baseline_per_n.values()) - valid_total:.6f}")

Checking for overlaps in best solutions...


Checking overlaps:   0%|          | 0/200 [00:00<?, ?it/s]

Checking overlaps:  17%|█▋        | 34/200 [00:00<00:00, 338.07it/s]

Checking overlaps:  34%|███▍      | 68/200 [00:00<00:01, 112.62it/s]

Checking overlaps:  44%|████▎     | 87/200 [00:01<00:01, 70.59it/s] 

...

Checking overlaps:  99%|█████████▉| 198/200 [00:09<00:00,  7.33it/s]

Checking overlaps: 100%|█████████▉| 199/200 [00:10<00:00,  7.27it/s]

Checking overlaps: 100%|██████████| 200/200 [00:10<00:00,  7.06it/s]

Checking overlaps: 100%|██████████| 200/200 [00:10<00:00, 19.44it/s]


Found 0 N values with overlaps: []...

Valid ensemble score: 70.647327
Improvement from baseline: 0.000000


In [7]:
# Override N=1 with optimal value (from jonathanchan kernel)
# x=0, y=0, deg=45 gives score 0.6612499...
optimal_n1 = pd.DataFrame({
    'id': ['001_0'],
    'x': ['s0.0'],
    'y': ['s0.0'],
    'deg': ['s45.0']
})

xs = [strip_s(v) for v in optimal_n1['x'].values]
ys = [strip_s(v) for v in optimal_n1['y'].values]
ds = [strip_s(v) for v in optimal_n1['deg'].values]
n1_score = score_group(xs, ys, ds)

print(f"Optimal N=1 score: {n1_score:.10f}")
print(f"Current N=1 score: {best_per_n[1]['score']:.10f}")

if n1_score < best_per_n[1]['score']:
    print(f"Improvement: {best_per_n[1]['score'] - n1_score:.10f}")
    best_per_n[1] = {
        'score': n1_score,
        'data': optimal_n1.copy(),
        'source': 'optimal_n1'
    }

# Final score
final_total = sum(d['score'] for d in best_per_n.values())
print(f"\nFinal ensemble score: {final_total:.6f}")

Optimal N=1 score: 0.6612500000
Current N=1 score: 0.6612500000

Final ensemble score: 70.647327


In [8]:
# Show which sources contributed
source_counts = {}
for n in range(1, 201):
    src = best_per_n[n]['source']
    source_counts[src] = source_counts.get(src, 0) + 1

print("Sources contributing to ensemble:")
for src, count in sorted(source_counts.items(), key=lambda x: -x[1]):
    print(f"  {src}: {count} N values")

Sources contributing to ensemble:
  valid_baseline: 198 N values
  ensemble.csv: 1 N values
  submission.csv: 1 N values


In [9]:
# Create the final submission
rows = []
for n in range(1, 201):
    rows.append(best_per_n[n]['data'])

final_df = pd.concat(rows, ignore_index=True)
final_df['sn'] = final_df['id'].str.split('_').str[0].astype(int)
final_df['si'] = final_df['id'].str.split('_').str[1].astype(int)
final_df = final_df.sort_values(['sn', 'si']).drop(columns=['sn', 'si'])
final_df = final_df[['id', 'x', 'y', 'deg']]

print(f"Final submission shape: {final_df.shape}")
print(f"Expected: (20100, 4)")

# Save
os.makedirs('/home/code/experiments/005_comprehensive_ensemble', exist_ok=True)
final_df.to_csv('/home/code/experiments/005_comprehensive_ensemble/submission.csv', index=False)

# Save metrics
metrics = {
    'cv_score': final_total,
    'baseline_score': sum(d['score'] for d in baseline_per_n.values()),
    'improvement': sum(d['score'] for d in baseline_per_n.values()) - final_total,
    'target': 68.888293,
    'gap': final_total - 68.888293,
    'num_sources': len(source_counts),
    'overlap_ns_reverted': len(overlap_ns)
}

with open('/home/code/experiments/005_comprehensive_ensemble/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print(f"\nMetrics: {metrics}")

Final submission shape: (20100, 4)
Expected: (20100, 4)

Metrics: {'cv_score': 70.6473268976368, 'baseline_score': 70.64732689763682, 'improvement': 1.4210854715202004e-14, 'target': 68.888293, 'gap': 1.7590338976367974, 'num_sources': 3, 'overlap_ns_reverted': 0}


In [10]:
# Final validation - check ALL N values for overlaps
print("Final validation - checking all N values...")

final_overlaps = []
for n in tqdm(range(1, 201), desc="Final validation"):
    g = best_per_n[n]['data']
    xs = [strip_s(v) for v in g['x'].values]
    ys = [strip_s(v) for v in g['y'].values]
    ds = [strip_s(v) for v in g['deg'].values]
    
    has_overlap, msg = check_overlaps(xs, ys, ds)
    if has_overlap:
        final_overlaps.append((n, msg))

if final_overlaps:
    print(f"\nWARNING: {len(final_overlaps)} N values still have overlaps!")
    for n, msg in final_overlaps[:10]:
        print(f"  N={n}: {msg}")
else:
    print("\n✓ All N values pass overlap validation!")
    print(f"\nFinal score: {final_total:.6f}")
    print(f"Gap to target: {final_total - 68.888293:.6f}")

Final validation - checking all N values...


Final validation:   0%|          | 0/200 [00:00<?, ?it/s]

Final validation:  18%|█▊        | 35/200 [00:00<00:00, 339.99it/s]

Final validation:  34%|███▍      | 69/200 [00:00<00:01, 110.53it/s]

Final validation:  44%|████▍     | 88/200 [00:01<00:01, 69.19it/s] 

...

Final validation:  99%|█████████▉| 198/200 [00:10<00:00,  7.08it/s]

Final validation: 100%|█████████▉| 199/200 [00:10<00:00,  7.04it/s]

Final validation: 100%|██████████| 200/200 [00:10<00:00,  7.02it/s]

Final validation: 100%|██████████| 200/200 [00:10<00:00, 19.39it/s]


✓ All N values pass overlap validation!

Final score: 70.647327
Gap to target: 1.759034


In [11]:
# Let's scan ALL snapshots more comprehensively
# The previous exp_004 found improvements from snapshots 21336527339, 21331543270, 21145966992

snapshot_base = '/home/nonroot/snapshots/santa-2025/'
snapshot_dirs = [d for d in os.listdir(snapshot_base) if os.path.isdir(os.path.join(snapshot_base, d))]
print(f"Found {len(snapshot_dirs)} snapshot directories")

# Find all submission.csv files in snapshots
snapshot_submissions = []
for snap_dir in snapshot_dirs:
    snap_path = os.path.join(snapshot_base, snap_dir)
    for root, dirs, files in os.walk(snap_path):
        for f in files:
            if f.endswith('.csv'):
                snapshot_submissions.append(os.path.join(root, f))

print(f"Found {len(snapshot_submissions)} CSV files in snapshots")

Found 114 snapshot directories
Found 3437 CSV files in snapshots


In [12]:
# Score all snapshot submissions and find best per-N
all_per_n_scores = {}  # {n: [(score, filepath, data), ...]}

for n in range(1, 201):
    all_per_n_scores[n] = []

# Start with baseline
for n in range(1, 201):
    all_per_n_scores[n].append((baseline_per_n[n]['score'], 'valid_baseline', baseline_per_n[n]['data']))

# Process all snapshot CSVs
for fp in tqdm(snapshot_submissions[:500], desc="Processing snapshots"):  # Limit to 500 for speed
    try:
        df = pd.read_csv(fp)
        if not {'id', 'x', 'y', 'deg'}.issubset(df.columns):
            continue
        if len(df) != 20100:
            continue
            
        df['N'] = df['id'].astype(str).str.split('_').str[0].astype(int)
        
        for n, g in df.groupby('N'):
            if n < 1 or n > 200:
                continue
            xs = [strip_s(v) for v in g['x'].values]
            ys = [strip_s(v) for v in g['y'].values]
            ds = [strip_s(v) for v in g['deg'].values]
            score = score_group(xs, ys, ds)
            
            # Only add if better than baseline
            if score < baseline_per_n[n]['score'] - 1e-8:
                all_per_n_scores[n].append((score, fp, g.drop(columns=['N']).copy()))
    except Exception as e:
        continue

# Count improvements
improvements = sum(1 for n in range(1, 201) if len(all_per_n_scores[n]) > 1)
print(f"\nFound improvements for {improvements} N values")

Processing snapshots:   0%|          | 0/500 [00:00<?, ?it/s]

Processing snapshots:   0%|          | 1/500 [00:00<00:59,  8.42it/s]

Processing snapshots:   0%|          | 2/500 [00:00<01:08,  7.31it/s]

Processing snapshots:   1%|          | 3/500 [00:00<01:02,  7.94it/s]

Processing snapshots:   1%|          | 4/500 [00:00<01:00,  8.14it/s]

...

Processing snapshots: 100%|█████████▉| 498/500 [01:01<00:00,  7.31it/s]

Processing snapshots: 100%|█████████▉| 499/500 [01:01<00:00,  7.53it/s]

Processing snapshots: 100%|██████████| 500/500 [01:01<00:00,  6.77it/s]

Processing snapshots: 100%|██████████| 500/500 [01:01<00:00,  8.10it/s]


Found improvements for 199 N values


In [13]:
# For each N with improvements, check if the best solution is valid (no overlaps)
best_valid_per_n = {n: baseline_per_n[n].copy() for n in range(1, 201)}

for n in tqdm(range(1, 201), desc="Finding best valid"):
    candidates = sorted(all_per_n_scores[n], key=lambda x: x[0])  # Sort by score
    
    for score, fp, data in candidates:
        xs = [strip_s(v) for v in data['x'].values]
        ys = [strip_s(v) for v in data['y'].values]
        ds = [strip_s(v) for v in data['deg'].values]
        
        has_overlap, msg = check_overlaps(xs, ys, ds)
        if not has_overlap:
            if score < best_valid_per_n[n]['score']:
                best_valid_per_n[n] = {
                    'score': score,
                    'data': data,
                    'source': os.path.basename(os.path.dirname(os.path.dirname(fp)))
                }
            break  # Found best valid, move to next N

# Calculate total
total_valid = sum(d['score'] for d in best_valid_per_n.values())
print(f"\nBest valid ensemble: {total_valid:.6f}")
print(f"Improvement from baseline: {sum(d['score'] for d in baseline_per_n.values()) - total_valid:.6f}")

Finding best valid:   0%|          | 0/200 [00:00<?, ?it/s]

Finding best valid:  12%|█▏        | 24/200 [00:00<00:00, 226.70it/s]

Finding best valid:  24%|██▎       | 47/200 [00:00<00:01, 83.87it/s] 

Finding best valid:  30%|███       | 60/200 [00:00<00:02, 57.03it/s]

Finding best valid:  34%|███▍      | 69/200 [00:01<00:02, 45.42it/s]

...

Finding best valid:  99%|█████████▉| 198/200 [00:15<00:00,  5.63it/s]

Finding best valid: 100%|█████████▉| 199/200 [00:15<00:00,  5.63it/s]

Finding best valid: 100%|██████████| 200/200 [00:15<00:00,  5.67it/s]

Finding best valid: 100%|██████████| 200/200 [00:15<00:00, 12.87it/s]


Best valid ensemble: 70.615744
Improvement from baseline: 0.031583


In [14]:
# exp_004 failed with overlaps in group 002 (N=2)\n# Let's check the N=2 solution in our current ensemble\n\n# Load the exp_004 submission\nexp004_df = pd.read_csv('/home/code/experiments/004_ensemble_valid/submission.csv')\nexp004_df['N'] = exp004_df['id'].astype(str).str.split('_').str[0].astype(int)\n\nn2_data = exp004_df[exp004_df['N'] == 2]\nprint(\"N=2 solution from exp_004:\")\nprint(n2_data)\n\nxs = [strip_s(v) for v in n2_data['x'].values]\nys = [strip_s(v) for v in n2_data['y'].values]\nds = [strip_s(v) for v in n2_data['deg'].values]\n\nprint(f\"\\nCoordinates:\")\nfor i in range(len(xs)):\n    print(f\"  Tree {i}: x={xs[i]:.15f}, y={ys[i]:.15f}, deg={ds[i]:.15f}\")\n\n# Check overlap with high precision\npolys = [get_tree_polygon(xs[i], ys[i], ds[i]) for i in range(len(xs))]\nfor i in range(len(polys)):\n    for j in range(i+1, len(polys)):\n        if polys[i].intersects(polys[j]):\n            inter = polys[i].intersection(polys[j])\n            print(f\"\\nTrees {i} and {j} intersect!\")\n            print(f\"  Intersection area: {inter.area:.20e}\")\n            print(f\"  touches: {polys[i].touches(polys[j])}\")"